In [6]:
import pandas as pd
import DataLoading
import NN_classes
import NN_evaluation
import torch
import matplotlib.pyplot as plt
import numpy as np
from time import perf_counter

In [7]:
sc = "sc01"
period = "2030"
case= "RTS24"
folder = f"../Data/{case}_DC_fy/"

all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
te_s = 0.3
val_s = 0.3
dfs_in, dfs_out, dfs_inter = DataLoading.load_data_ext_out(folder, all_executions, period, sc, ["PowerFlow"],"SystemCosts")
dfs_inter_j = DataLoading.join_frames_inter_layer(dfs_inter,all_executions)
dfs_inter_j = DataLoading.trim_columns_to_common(dfs_inter_j)
# Convert to pytorch tensors
ts_in, ts_out, ts_inter = DataLoading.split_tr_val_te_ext_out(dfs_in, dfs_out, dfs_inter_j, all_executions, te_s, val_s)
d_ft_in, d_ft_out, d_ft_inter,maxs = DataLoading.concat_and_normalize_ext_out(ts_in, ts_out, ts_inter, all_executions)

input_size = dfs_in["Network_Existing_Generation_Full"].shape[1]
inter_size = dfs_inter_j["Network_Existing_Generation_Full"].shape[1]

t_in_fy, t_out_fy, t_inter_fy, maxs = DataLoading.concat_all_exec_fy(dfs_in=dfs_in, dfs_out=dfs_out, dfs_inter_j=dfs_inter_j,executions=all_executions,normalize_out=True)


input_f_sc01_Network_Existing_Generation_Full_2030.csv
90
input_f_sc01_Network_Full_Generation_Full_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_101_N_102_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_101_N_103_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_101_N_105_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_102_N_104_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_102_N_106_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_103_N_109_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_103_N_124_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_104_N_109_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_105_N_110_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_106_N_108_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_106_N_110_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_107_N_108_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_108_N_109_cac1_2030.csv
90
input_f_sc01_PINT_Network_Line_In_N_108_N_110_cac1_2030.csv
90
input_

In [8]:
sc = "sc01"
period = "2030"
#folder = "../Data/3-bus_AC_fy_ext_o_dummy_LCOE/"
#folder = "../Data/9n_AC_fy_ext_o_dummy_LCOE/"
folder = f"../Data/{case}_DC_fy/"

all_executions = DataLoading.list_executions(folder=folder,per = period,sc=sc)
dfs_in_fy = DataLoading.load_data_input_only(folder,all_executions,period,sc)

input_size = dfs_in["Network_Existing_Generation_Full"].shape[1]
inter_size = dfs_inter_j["Network_Existing_Generation_Full"].shape[1]

AttributeError: module 'DataLoading' has no attribute 'load_data_input_only'

In [8]:
#hyperloop_name = "3-bus_AC_12w_dummy_0.3_v0.4_PF_LCOE_0_10"
#hyperloop_name = "9n_AC_12w_dummy_0.3_v0.4_PF_LCOE_0_37"
hyperloop_name = "RTS24_AC_12w_dummy_0.3_v0.4_PF_LCOE_0_40"
hyperloop_name = f"Decay_rand_Weeks_{case}_DC_0.25_v0.33_PF_0_74"
hyperloop_name = "Decay_2016h_rand_Days_RTS24_DC_0.25_v0.33_PF_0_80"
folder_exports = f"{case}_DC_fy"


#df_losses = pd.read_csv(f"ResultsClusterRuns/Loss_results_csv/{hyperloop_name}.csv",index_col=0)
df_losses = pd.read_csv(f"Loss_results_csv/{hyperloop_name}.csv",index_col=0)


In [9]:
df_losses

,Model_type,Dor,Relu_out,Batch_size,Lr,Lri,Lrs,Lrg,Epochs,Min_val,...,fy_l_mse,fy_l_mae,fy_l_mae_man,Train_time,Eval_time,alpha,beta,MAE,Test size,Val size
0,"(3, 1)",0.0,False,128,"(0.0025, 32, 1)",0.0025,32,1.00,128,min_val,...,0.000028,0.001005,0.001005,350.653530,0.044580,0.0,1,False,0.25,0.333333
1,"(3, 1)",0.0,False,128,"(0.0025, 32, 1)",0.0025,32,1.00,128,all_epochs,...,0.000029,0.001040,0.001040,350.653530,0.038112,0.0,1,False,0.25,0.333333
2,"(3, 1)",0.0,False,128,"(0.0025, 32, 1)",0.0025,32,1.00,128,min_val,...,0.000027,0.000193,0.000193,353.884886,0.037574,0.0,1,True,0.25,0.333333
3,"(3, 1)",0.0,False,128,"(0.0025, 32, 1)",0.0025,32,1.00,128,all_epochs,...,0.000033,0.000354,0.000354,353.884886,0.031234,0.0,1,True,0.25,0.333333
4,"(3, 1)",0.0,False,128,"(0.0025, 32, 1)",0.0025,32,1.00,128,min_val,...,0.000039,0.001551,0.001551,343.767072,0.039587,0.2,1,False,0.25,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"(3, 1)",0.1,False,128,"(0.0025, 32, 0.25)",0.0025,32,0.25,128,all_epochs,...,0.000068,0.002671,0.002671,377.420342,0.036169,0.0,1,True,0.25,0.333333
60,"(3, 1)",0.1,False,128,"(0.0025, 32, 0.25)",0.0025,32,0.25,128,min_val,...,0.000064,0.003284,0.003284,367.563164,0.041415,0.2,1,False,0.25,0.333333
61,"(3, 1)",0.1,False,128,"(0.0025, 32, 0.25)",0.0025,32,0.25,128,all_epochs,...,0.000070,0.003548,0.003548,367.563164,0.039169,0.2,1,False,0.25,0.333333
62,"(3, 1)",0.1,False,128,"(0.0025, 32, 0.25)",0.0025,32,0.25,128,min_val,...,0.000051,0.002134,0.002134,373.955389,0.036603,0.2,1,True,0.25,0.333333


In [12]:
df_extraction_time = pd.DataFrame()
i=0
for loss in ["Te_l_mse"]:
    for xth_best in [1]:
        ##Find and: load model
        for alpha in [0.0,0.16]:
            #f = df_losses.alpha ==alpha
            f = ((df_losses.Nb_hours_used) <=2050) & (df_losses.Sel_method == "Days")
            
            # f = df_losses.Model_type == "(3, 2)"
            loss_to_sort = loss
            
            

            row = NN_evaluation.find_xthbest_model_params_from_df(df_losses[f],loss_to_sort,xth_best)
            if row.Sel_method.item() == "Hours": 
                hyperloop_name = f"Decay_rand_Hours_{case}_DC_0.25_v0.33_PF_0_20"
            elif row.Sel_method.item() == "Days":
                hyperloop_name = f"Decay_rand_Days_{case}_DC_0.25_v0.33_PF_0_74"
                hyperloop_name = "Decay_2016h_rand_Days_RTS24_DC_0.25_v0.33_PF_0_80"
            else: 
                print("Problem")

            m = NN_evaluation.create_model_and_load_state_from_row(row,input_size,inter_size,hyperloop_name,cluster_run=False,hidden_sizes = [60,60,60,38,19])

            ##Use model for estimates
            df_all_exec = pd.DataFrame()
            for ex in all_executions:
                t_start = perf_counter()
                prediction = NN_evaluation.get_NN_estimates_from_dfs_in(m,ex,dfs_in,maxs)
                t_elapsed = perf_counter() - t_start
                #actual= get_actual(ex,dfs_out) 
                df_all_exec[ex] = prediction

            mpd = model_params_dict = NN_evaluation.extract_model_params_from_row(row)
            keys = ["Model_type","nb_e","lr","dor","ro","bs","alpha","MAE", "Min_val"]
            name = f"Estimated_OC_fy_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}ro_{mpd[keys[5]]}bs_{mpd[keys[6]]}ill_{mpd[keys[7]]}MAE_{mpd[keys[8]]}Min_val"
            #name = f"Estimated_OC_fy_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE"
            time_row = pd.DataFrame({"Model":name,"Evaluation Time": t_elapsed},index = [i] )
            i+=1
            df_extraction_time = pd.concat([df_extraction_time,time_row])
            #Write exports to csv
            df_all_exec.to_csv(f"NN_estimate_exports/{folder_exports}/{name}.csv")
df_extraction_time.to_csv(f"NN_estimate_exports/{folder_exports}/evaluation_times.csv")

[60, 60, 60, 38, 19]
OE_2016hours_(3, 1)h_128e_(0.0025, 32, 0.25)lr_0dor_Falsero_128bs_0ill_TrueMAE all_epochs
trained_models/Decay_2016h_rand_Days_RTS24_DC_0.25_v0.33_PF_0_80/all_epochs/model_OE_2016hours_(3, 1)h_128e_(0.0025, 32, 0.25)lr_0dor_Falsero_128bs_0ill_TrueMAE.pth


C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.recipro

C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.reciprocal() * other
C:\Workdir\Programs\Miniconda\envs\jr23\lib\site-packages\torch\_tensor.py:852: RuntimeWarning: invalid value encountered in multiply
  return self.recipro

OSError: Cannot save file into a non-existent directory: 'NN_estimate_exports\RTS24_DC_fy'

In [7]:
ex = all_executions[0]

df_all_exec = pd.DataFrame()
for ex in all_executions:
    prediction = get_NN_estimates_from_dfs_in(m,ex,dfs_in)
    #actual= get_actual(ex,dfs_out) 
    df_all_exec[ex] = prediction

mpd = model_params_dict = extract_model_params_from_row(row)
keys = ["Model_type","nb_e","lr","dor","np","ro","bs","alpha","MAE"]
name = f"Estimated_OC_12w_{mpd[keys[0]]}h_{mpd[keys[1]]}e_{mpd[keys[2]]}lr_{mpd[keys[3]]}dor_{mpd[keys[4]]}np_{mpd[keys[5]]}ro_{mpd[keys[6]]}bs_{mpd[keys[7]]}ill_{mpd[keys[8]]}MAE"
df_all_exec.to_csv(f"NN_estimate_exports/{name}.csv")

IndexError: index 0 is out of bounds for axis 0 with size 0